Different Bios testing

In [ ]:
import sys
sys.path.append('../pyCRLD')


In [32]:
import numpy as np
import matplotlib.pyplot as plt
import copy

from pyCRLD.Environments.SocialDilemma import SocialDilemma
from pyCRLD.Environments.EcologicalPublicGood import EcologicalPublicGood

from pyCRLD.Agents.StrategyActorCritic import stratAC
from pyCRLD.Agents.POStrategyActorCritic import POstratAC


from pyCRLD.Utils import FlowPlot as fp
from fastcore.utils import *
from jax import jit
import jax.numpy as jnp
from pyCRLD.Environments.HistoryEmbedding import HistoryEmbedded

from scipy.stats import kstest

from scipy.stats import qmc
import itertools as it
import pandas as pd

global_seed = 40
np.random.seed(global_seed)

In [42]:
from pyCRLD.Agents.test import hello

hello_test = hello()
hello_test.hello()


Hello World


In [10]:
socdi = SocialDilemma(R=1.0, T=1.2, S=-0.5, P=0.0)
memo1pd = HistoryEmbedded(socdi, h = (1, 1, 1))

mae_strat = stratAC(env=memo1pd, learning_rates=0.1, discount_factors=0.99)
mae_postrat = POstratAC(env=memo1pd, learning_rates=0.1, discount_factors=0.99)

In [ ]:
mae_postrat = POstratAC(env=memo1pd, learning_rates=0.1, discount_factors=0.9)

policy_test = np.round(mae_postrat.random_softmax_policy(),1)
print(policy_test)

e  = 10e-10
state_dist_test = np.array([0.5, 0.5, 0, 0])
state_dist_test_with_epsilon = (state_dist_test + e)/np.sum(state_dist_test + e)

print(state_dist_test_with_epsilon)


state_dist_test_with_epsilon_temp = np.array([0.5, 0.5, e, e])

state_dist_test[state_dist_test == 0] = e


[[[0.3        0.7       ]
  [0.2        0.8       ]
  [0.5        0.5       ]
  [0.3        0.7       ]]

 [[0.1        0.90000004]
  [1.         0.        ]
  [0.7        0.3       ]
  [0.1        0.90000004]]]
[4.99999e-01 4.99999e-01 9.99996e-07 9.99996e-07]


In [28]:
def _bios_with_epsilon(O, pS):
        i, s, o = 0, 1, 2 # variables 

        e  = 10e-10
        pS[pS == 0] = e
        b = jnp.einsum(O, [i,s,o], pS, [s], [i,s,o])
        bsum = b.sum(axis=1, keepdims=True)
        print(bsum)
        bsum = bsum + (bsum == 0)  # to avoid dividing by zero
        Biso = b / bsum
        Bios = jnp.swapaxes(Biso, 1,-1)
        
        return Bios

bios_test = _bios_with_epsilon(mae_postrat.O, state_dist_test)
print(bios_test)

[[[5.e-01 5.e-01 1.e-09 1.e-09]]

 [[5.e-01 5.e-01 1.e-09 1.e-09]]]
[[[1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 1. 0.]
  [0. 0. 0. 1.]]

 [[1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 1. 0.]
  [0. 0. 0. 1.]]]


In [ ]:
def _bios(self, X, pS):
        i, s, o = 0, 1, 2 # variables 


        b = jnp.einsum(self.O, [i,s,o], pS, [s], [i,s,o], optimize=self.opti)
        bsum = b.sum(axis=1, keepdims=True)
        bsum = bsum + (bsum == 0)  # to avoid dividing by zero
        Biso = b / bsum
        Bios = jnp.swapaxes(Biso, 1,-1)
        
        return Bios
        
def fast_Bios(self, X):
        """
        Compute 'belief' that environment is in stats s given agent i
        observes observation o (Bayes Rule)
        
        Unsafe when stationary state distribution is not unique
        (i.e., when policies are too extreme)
        """
        i, s, o = 0, 1, 2 # variables 
        # pS = self.statedist(X) # from full obs base (requires Tss from above)
        pS = self._jaxPs(X, self._last_statedist)
        

        b = jnp.einsum(self.O, [i,s,o], pS, [s], [i,s,o], optimize=self.opti)
        bsum = b.sum(axis=1, keepdims=True)
        bsum = bsum + (bsum == 0)  # to avoid dividing by zero
        Biso = b / bsum
        Bios = jnp.swapaxes(Biso, 1,-1)

In [29]:
def _bios_with_epsilon(self, X, pS):
        i, s, o = 0, 1, 2 # variables 

        pS[pS == 0] = 10e-10


        b = jnp.einsum(self.O, [i,s,o], pS, [s], [i,s,o], optimize=self.opti)
        bsum = b.sum(axis=1, keepdims=True)
        bsum = bsum + (bsum == 0)  # to avoid dividing by zero
        Biso = b / bsum
        Bios = jnp.swapaxes(Biso, 1,-1)
        
        return Bios

   

def fast_Bios_with_epsilon(self, X):
        """
        Compute 'belief' that environment is in stats s given agent i
        observes observation o (Bayes Rule)
        
        Unsafe when stationary state distribution is not unique
        (i.e., when policies are too extreme)
        """
        i, s, o = 0, 1, 2 # variables 
        # pS = self.statedist(X) # from full obs base (requires Tss from above)
        pS = self._jaxPs(X, self._last_statedist)
        
        pS[pS == 0] = 10e-10
        b = jnp.einsum(self.O, [i,s,o], pS, [s], [i,s,o], optimize=self.opti)
        bsum = b.sum(axis=1, keepdims=True)
        bsum = bsum + (bsum == 0)  # to avoid dividing by zero
        Biso = b / bsum
        Bios = jnp.swapaxes(Biso, 1,-1)

### Experiments and Observations 